In [26]:
import tensorflow as tf
import numpy as np

#하이퍼 파라미터 설정

In [27]:
epochs = 1000


#네트워크 구조 정의

In [28]:
#입력 2, 은닉 128(sigmoid), 출력 10(softmax)
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel,self).__init__()
    self.d1 = tf.keras.layers.Dense(128,input_dim=2,activation='sigmoid')#dense가 fully connected
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')#input은 위에서 해서 생략가능, 위에도 솔직히 노 상관 들어올때 정의됨


  def __call__(self,x, training = None, mask = None):
    x = self.d1(x)
    return self.d2(x)

#학습 루프 정의

In [29]:
@tf.function #데코
#auto graph할거임
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_metric):
  with tf.GradientTape() as tape: 
    #이 내부에선 모두 계산되어 tape에 들어가게 된다.
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)#loss를 뒤에 var로 미분한다. grad(loss), df(x)/x
  #편미분된게 gradients에 넣어지고 이걸 적용
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))#optimizer를 통해 적용
  #평가지표
  train_loss(loss)
  train_metric(labels, predictions)


#데이터셋 생성, 전처리

In [30]:
np.random.seed(0)
#랜덤 시드를 고정시켜둔당

pts = list()#입력값, 두가지 스칼라
labels = list()#출력값, 10가지 클래스를 표현하는 sparse

center_pts = np.random.uniform(-8.0,8.0,(10,2))
for label, center_pt in enumerate(center_pts):
  for _ in range(100):
    pts.append(center_pt + np.random.randn(*center_pt.shape))
    labels.append(label)

pts = np.stack(pts, axis = 0).astype(np.float32) #stack쓰면 numpy object됨
#gpu는 기본적으로 float32를 쓴다고 생각하면 된다.
labels = np.stack(labels,axis =0 )

train_ds = tf.data.Dataset.from_tensor_slices((pts,labels)).shuffle(1000).batch(32) #합쳐주고 iterable한 오브젝트가 됨



#모델 생성

In [31]:
model = MyModel()

#손실 함수 및 최적화 알고리즘 설정

In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()#sparse면 정수로 몇 번째가 1이냐!!
optimizer = tf.keras.optimizers.Adam()

#평가지표 설정

In [33]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')#다 맞추면 1 아니면 0 이런식임


#학습 루프

In [35]:
for epoch in range(epochs):
  for x, label in train_ds:#batch = 32니까 32개따리
    train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)

  template = 'epoch {}, loss {}, acc {}'
  print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          ))

epoch 1, loss 0.32526034116744995, acc 88.3494644165039
epoch 2, loss 0.3250965476036072, acc 88.35274505615234
epoch 3, loss 0.32492294907569885, acc 88.35529327392578
epoch 4, loss 0.3247580826282501, acc 88.35612487792969
epoch 5, loss 0.3246265649795532, acc 88.35816955566406
epoch 6, loss 0.3244670331478119, acc 88.36019897460938
epoch 7, loss 0.3243151903152466, acc 88.36221313476562
epoch 8, loss 0.3241491913795471, acc 88.36543273925781
epoch 9, loss 0.3239864706993103, acc 88.3671875
epoch 10, loss 0.32384347915649414, acc 88.36941528320312
epoch 11, loss 0.32369062304496765, acc 88.37330627441406
epoch 12, loss 0.3235221803188324, acc 88.3759765625
epoch 13, loss 0.3233511447906494, acc 88.37935638427734
epoch 14, loss 0.3231958746910095, acc 88.38177490234375
epoch 15, loss 0.32303890585899353, acc 88.3841781616211
epoch 16, loss 0.32289350032806396, acc 88.38727569580078
epoch 17, loss 0.32273516058921814, acc 88.38823699951172
epoch 18, loss 0.32258138060569763, acc 88.391

#데이터셋 및 학습 파라미터 저장

In [37]:
np.savez_compressed('ch2data_fromjoonkee.npz',inputs=pts,labels=labels) #savez를 써서 여러개, compressed를 써서 압축

W_h,b_h = model.d1.get_weights()
W_o,b_o = model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ddd.npz',
                    W_h=W_h,
                    W_o=W_o,
                    b_h=b_h,
                    b_o=b_o
                    )
